In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path
from compile_sweep_results import compile_sweep_results

In [ ]:
root = Path("/media/nick/hdd021/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/symmetry_breaking/pde/sweeps/")
sweep_name = "sweep01_neutralization_v2"
df = compile_sweep_results(output_dir=root / sweep_name)
print(df.shape)
df.head()

In [19]:
fig = px.scatter(df, x="roi_frac", y="roi_fold_change", log_y=True)
# add a line at y=1
fig.add_hline(y=1, line_dash="dash", line_color="white", annotation_text="y=1", annotation_position="top left")
# add vertical line at x=500/3500
fig.add_vline(x=500/3500, line_dash="dash", line_color="white", annotation_text="x=500/3500", annotation_position="top right")
# set axis limits
fig.update_xaxes(title_text="Fraction of ROI", range=[0, 1])
fig.update_yaxes(title_text="Max Fold Change", range=[-4, 4])

fig.show()

In [21]:
df["sim_id"] = df["__source_file__"].str.replace("result_", "").str.replace(".json", "")

In [22]:
param_cols = ["sim_id", 'K_NL', 'K_A', 'K_R', 'K_I', 'mu_L', 'N_amp', 'N_sigma', 'sigma_N',
       'sigma_L', 'D0_N', 'D0_L', 'alpha_L',
       'alpha_N', 'tau_rho', 'n', 'm', 'p', 'q', 'roi_frac', 'roi_fold_change',
       'max_fold_change']
param_df = df[param_cols].copy()

nodal_df = df[["sim_id", 'profile_Activator']].copy()
lefty_df = df[["sim_id", 'profile_Repressor']].copy()
rho_df = df[["sim_id", 'profile_rho']].copy()

# save
param_df.to_csv(root / f"{sweep_name}_params_df.csv", index=False)
nodal_df.to_csv(root / f"{sweep_name}_nodal_df.csv", index=False)
lefty_df.to_csv(root / f"{sweep_name}_lefty_df.csv", index=False)
rho_df.to_csv(root / f"{sweep_name}_rho_df.csv", index=False)

In [17]:
ind = np.where((df["max_fold_change"] > 10) & (df["roi_frac"] > 0.9))[0]
df.loc[ind, "sigma_L"].unique()

array([ 5.17947468, 10.        ])

In [13]:
nodal_profile = np.asarray(nodal_df.loc[ind[0], "profile_Activator"])
fig = px.scatter(x=np.arange(len(nodal_profile)), y=nodal_profile)
fig.show()

In [46]:
param_df.loc[ind]

,sim_id,K_NL,K_A,K_R,K_I,mu_L,N_amp,N_sigma,sigma_N,sigma_L,...,alpha_L,alpha_N,tau_rho,n,m,p,q,roi_frac,roi_fold_change,max_fold_change
9002,98e30a02,77.426368,2154.43469,46.415888,359.381366,0.000293,10000.0,100.0,10.0,0.1,...,0.46,4.495556,3600,2,1,2,2,0.851788,3.680017,1.826367


In [15]:
df.loc[ind[0], "__source_file__"]

'result_4fe1390e.json'